In [1]:
import logging
import sys

from copy import deepcopy
from operator import add

import numpy as np
import pandas as pd

from numpy.lib.recfunctions import append_fields

from camelup import camelup, config
from camelup import utilities as util

In [2]:
def coins_to_numpy(game):
    return np.array(
        [(key[0], key[1]["coins"]) for key in [*game.player_dict.items()]],
        dtype=[("player", float), ("coins", float)],
    )


def bet_tiles_to_numpy(game):
    return np.array(
        [
            (key[0], camel[0], sum(camel[1]), len(camel[1]))
            for key in [*game.player_dict.items()]
            for camel in [*key[1]["bet_tiles"].items()]
        ],
        dtype=[("player", float), ("camel", "U10"), ("value", float), ("bets", float)],
    )


def create_prob_array(result, iter):
    bins, counts = np.unique(result["camel"], return_counts=True)
    counts_array = np.array(counts, dtype=[("counts", float)])
    result_array = append_fields(bins, "counts", counts_array["counts"], usemask=False)
    result_array.dtype.names = ("camel", "counts")
    prob_array = np.array(result_array["counts"] / iter, dtype=[("prob", float)])
    return append_fields(result_array, "prob", prob_array["prob"], usemask=False)


def turn_prob_numpy(game, iter):
    winner_result, tile_points_result = game.turn_monte(
        game.camel_dict, game.tiles_dict, iter=iter
    )
    prob_first = create_prob_array(winner_result[winner_result["place"] == 1], iter)
    prob_second = create_prob_array(winner_result[winner_result["place"] == 2], iter)
    prob_other = create_prob_array(winner_result[winner_result["place"] > 2], iter)
    exp_tile_points = util.numpy_group_by_sum(tile_points_result, "player", "points")
    return prob_first, prob_second, prob_other, exp_tile_points


def game_prob_numpy(game, iter):
    result = game.game_monte(game.camel_dict, game.tiles_dict, iter=iter)
    loser_place = max(result["place"])
    prob_first = create_prob_array(result[result["place"] == 1], iter)
    prob_last = create_prob_array(result[result["place"] == loser_place], iter)
    prob_other = create_prob_array(
        result[(result["place"] != 1) & (result["place"] != loser_place)], iter
    )
    return prob_first, prob_last, prob_other


def winner_loser_bets_to_numpy(game):
    winner_bets = np.array(
        game.winner_bets, dtype=[("player", float), ("camel", "U10")]
    )
    loser_bets = np.array(game.loser_bets, dtype=[("player", float), ("camel", "U6")])
    return winner_bets, loser_bets


In [3]:
game = camelup.Game(3)

camel_dict = {
    "red": {"height": 1, "space": 1, "need_roll": True},
    "blue": {"height": 2, "space": 1, "need_roll": True},
    "green": {"height": 1, "space": 2, "need_roll": True},
}
game.camel_dict = camel_dict

game.play_winner_card("blue")
game.play_loser_card("red")
game.play_bet_tile("blue")

game.state = 2
game.play_bet_tile("blue")
game.play_bet_tile("green")
game.play_winner_card("green")

In [4]:
iter = 1000

In [5]:
coins = coins_to_numpy(game)
turn_prob_first, turn_prob_second, turn_prob_other, exp_tile_points = turn_prob_numpy(
    game, iter
)
game_prob_first, game_prob_last, game_prob_other = game_prob_numpy(game, iter)
winner_bets, loser_bets = winner_loser_bets_to_numpy(game)
bet_tiles = bet_tiles_to_numpy(game)

In [6]:
bet_tiles['camel']

array(['blue', 'blue', 'green'], dtype='<U10')

In [7]:
turn_prob_first['camel']

array(['blue', 'green', 'red'], dtype='<U6')

In [8]:
def numpy_left_join(df1, df2, key):
    """
    Basic left join, return left join of 2 dataframe, duplicates allowing in df1. Only one key allowed
    """
    df2_descr = list(filter(lambda x: x[0] != key, df2.dtype.descr))
    new_df_dtype = np.dtype(list(set(df1.dtype.descr + df2_descr)))
    new_df = np.zeros(df1.shape, dtype=new_df_dtype)
    for col in df1.dtype.names:
        new_df[col] = df1[col]
    for new_row in new_df:
        for row in df2:
            if row[key]==new_row[key]:
                for col in df2.dtype.names:
                    new_row[col] = row[col]
                next
    return new_df

In [9]:
%%time
bets_pd = pd.DataFrame(bet_tiles).merge(
    pd.DataFrame(turn_prob_first), on="camel", how="left", suffixes=("", "_first")
)

CPU times: user 17.3 ms, sys: 1.74 ms, total: 19 ms
Wall time: 18.7 ms


In [10]:
%%time
bets_np = numpy_left_join(bet_tiles, turn_prob_first, 'camel')

CPU times: user 282 µs, sys: 1 µs, total: 283 µs
Wall time: 292 µs


In [11]:
bet_tiles

array([(1., 'blue', 5., 1.), (2., 'blue', 3., 1.), (2., 'green', 5., 1.)],
      dtype=[('player', '<f8'), ('camel', '<U10'), ('value', '<f8'), ('bets', '<f8')])

In [12]:
bets_pd

,player,camel,value,bets,counts,prob
0,1.0,blue,5.0,1.0,486.0,0.486
1,2.0,blue,3.0,1.0,486.0,0.486
2,2.0,green,5.0,1.0,314.0,0.314


In [13]:
from numpy.lib.recfunctions import append_fields

In [14]:
%%time
bets_np = append_fields(bets_np, 'exp_val', bets_np['value']*bets_np['prob'], '<f8', usemask=False)

CPU times: user 1.65 ms, sys: 548 µs, total: 2.2 ms
Wall time: 22.5 ms


In [15]:
%%time
bets_pd['exp_val']=bets_pd['value'] * bets_pd['prob']

CPU times: user 4.14 ms, sys: 1.88 ms, total: 6.02 ms
Wall time: 12.6 ms


In [16]:
def numpy_group_by_sum(array, index, sum_col):
    unique_groups = np.unique(array[index])
    sums = []
    for group in unique_groups:
        sums.append((group, array[array[index] == group][sum_col].sum()))
    return np.array(sums, dtype=[(f"{index}", float), (f"{sum_col}", float)])

In [17]:
%%time
numpy_group_by_sum(bets_np, 'player', 'exp_val')

CPU times: user 639 µs, sys: 291 µs, total: 930 µs
Wall time: 706 µs


array([(1., 2.43 ), (2., 3.028)],
      dtype=[('player', '<f8'), ('exp_val', '<f8')])

In [18]:
%%time
bets_pd.groupby('player')['exp_val'].sum()

CPU times: user 5.12 ms, sys: 1.75 ms, total: 6.87 ms
Wall time: 5.83 ms


player
1.0    2.430
2.0    3.028
Name: exp_val, dtype: float64

In [19]:
turn_prob_first.dtype.names

('camel', 'counts', 'prob')

In [20]:
def calc_exp_value_np(df, exp_value):
    multiply_array = df['prob'] * df['points']
    df = add_col_np(df, exp_value, multiply_array)
    return util.numpy_group_by_sum(df, 'player', exp_value)

def rename(df, columns, suffix):
    names = ()
    for col in df.dtype.names:
        if col in columns:
            names += (f'{col}_{suffix}',)
        else:
            names += (col,)
    df.dtype.names = names

def add_col_np(df, target_col, array):
    df = append_fields(df, target_col, array, '<f8', usemask=False)
    return df

In [21]:
def game_merge_to_final(final_df, game_df, suffix):
    return final_df.merge(
        game_df.groupby("player")["exp_value"].sum().reset_index(),
        on="player",
        how="left",
        suffixes=("", f"_{suffix}"),
    )

def calc_exp_value(df):
    df["exp_value"] = df["prob"] * df["points"]

## Testing the two against eachother

In [22]:
def calc_utility(game, iter):
    coins = coins_to_numpy(game)
    turn_prob_first, turn_prob_second, turn_prob_other, exp_tile_points = turn_prob_numpy(
        game, iter
    )
    game_prob_first, game_prob_last, game_prob_other = game_prob_numpy(game, iter)
    winner_bets, loser_bets = winner_loser_bets_to_numpy(game)
    bet_tiles = bet_tiles_to_numpy(game)
    rename(turn_prob_first, ['counts','prob'],'first')
    rename(turn_prob_second, ['counts','prob'],'second')
    rename(turn_prob_other, ['counts','prob'],'other')
    bets = numpy_left_join(bet_tiles, turn_prob_first, 'camel')
    bets = numpy_left_join(bets, turn_prob_second, 'camel')
    bets = numpy_left_join(bets, turn_prob_other, 'camel')
    multiply_array = (bets['value'] * bets['prob_first']) + (bets['bets'] * bets['prob_second']) - (bets['bets'] * bets['prob_other'])
    bets = add_col_np(bets, 'exp_value', multiply_array)
    bets_groupby = numpy_group_by_sum(bets, 'player', 'exp_value')
    final = numpy_left_join(coins, exp_tile_points, 'player')
    final = numpy_left_join(final, bets_groupby, 'player')
    game_first = numpy_left_join(winner_bets, game_prob_first, 'camel')
    game_last = numpy_left_join(loser_bets, game_prob_last, 'camel')
    game_winner_other = numpy_left_join(winner_bets, game_prob_other, 'camel')
    game_loser_other = numpy_left_join(loser_bets, game_prob_other, 'camel')
    game_first = add_col_np(game_first, 'points', config.BET_SCALING[0:game_first.shape[0]])
    game_last = add_col_np(game_last, 'points', config.BET_SCALING[0:game_last.shape[0]])
    game_winner_other = add_col_np(game_winner_other, 'points', [1]*game_winner_other.shape[0])
    game_loser_other = add_col_np(game_loser_other, 'points', [1]*game_loser_other.shape[0])
    final = numpy_left_join(final, calc_exp_value_np(game_first, 'exp_value_first'), 'player')
    final = numpy_left_join(final, calc_exp_value_np(game_last, 'exp_value_last'), 'player')
    final = numpy_left_join(final, calc_exp_value_np(game_winner_other, 'exp_value_winner_other'), 'player')
    final = numpy_left_join(final, calc_exp_value_np(game_loser_other, 'exp_value_loser_other'), 'player')
    multiply_array = (final["coins"]
        + final["points"]
        + final["exp_value"]
        + final["exp_value_first"]
        + final["exp_value_last"]
        - final["exp_value_winner_other"]
        - final["exp_value_loser_other"]
    )
    final = add_col_np(final, 'utility', multiply_array)

todo fix case where there is only 1 outcome

In [23]:
    turn_prob_first, turn_prob_second, turn_prob_other, exp_tile_points = turn_prob_numpy(
        game, 1
    )

In [24]:
turn_prob_first

array([('blue', 1., 1.)],
      dtype=[('camel', '<U4'), ('counts', '<f8'), ('prob', '<f8')])

In [25]:
bet_tiles

array([(1., 'blue', 5., 1.), (2., 'blue', 3., 1.), (2., 'green', 5., 1.)],
      dtype=[('player', '<f8'), ('camel', '<U10'), ('value', '<f8'), ('bets', '<f8')])

In [26]:
turn_prob_other

array([('red', 1., 1.)],
      dtype=[('camel', '<U3'), ('counts', '<f8'), ('prob', '<f8')])

In [27]:
numpy_left_join(bet_tiles, turn_prob_other, 'camel')

array([('blue', 0., 1., 1., 5., 0.), ('blue', 0., 2., 1., 3., 0.),
       ('green', 0., 2., 1., 5., 0.)],
      dtype=[('camel', '<U10'), ('counts', '<f8'), ('player', '<f8'), ('bets', '<f8'), ('value', '<f8'), ('prob', '<f8')])

In [28]:
def parse_dump(string):
    """Gets information from the AST dump and return the AST object and the id

    Parameters
    ----------
    string : str
        Tree to extract the id for

    Returns
    -------
    str
        A string with the object and name info

    """
    re1 = "id='(.*?)',"
    return string.split('(')[0] + ' - ' + re.search(re1, string).group(1)


def create_benchmark_func(tree, performance):
    """Creating a benchmark function named benchmark_`func`

    Parameters
    ----------
    tree : str
        AST for function to create benchmark function for
    performance : dict
        Dictionary to capture function

    Returns
    -------
    str :
        Tree of the benchmark function

    """
    t = 0
    new_first_layer = list()
    for node in tree.body[0].body:
        if node.__class__ != ast.Return:
            node_dump = ast.dump(node)
            parsed_dump = parse_dump(node_dump)
            performance[parsed_dump] = list()
            start_time = ast.parse(f"t{t} = time()").body[0]
            t+=1
            end_time = ast.parse(f"t{t} = time()").body[0]
            dif = ast.parse(f"performance['{parsed_dump}'].append(t{t}-t{t-1})").body[0]
            new_first_layer.extend([start_time, node, end_time, dif])
            t+=1
        else:
            new_first_layer.extend([node])
    tree.body[0].body = new_first_layer
    tree.body[0].name = f'benchmark_{tree.body[0].name}'
    return tree


In [29]:
from copy import deepcopy
from time import time
import numpy as np
import inspect
import ast
from io import StringIO
import re

In [30]:
func_str = inspect.getsource(calc_utility)
tree = ast.parse(StringIO(func_str).read())
global performance
performance = dict()
exec(compile(create_benchmark_func(tree, performance), filename="<ast>", mode="exec"))
print(ast.dump(tree))

Module(body=[FunctionDef(name='benchmark_calc_utility', args=arguments(args=[arg(arg='game', annotation=None), arg(arg='iter', annotation=None)], vararg=None, kwonlyargs=[], kw_defaults=[], kwarg=None, defaults=[]), body=[Assign(targets=[Name(id='t0', ctx=Store())], value=Call(func=Name(id='time', ctx=Load()), args=[], keywords=[])), Assign(targets=[Name(id='coins', ctx=Store())], value=Call(func=Name(id='coins_to_numpy', ctx=Load()), args=[Name(id='game', ctx=Load())], keywords=[])), Assign(targets=[Name(id='t1', ctx=Store())], value=Call(func=Name(id='time', ctx=Load()), args=[], keywords=[])), Expr(value=Call(func=Attribute(value=Subscript(value=Name(id='performance', ctx=Load()), slice=Index(value=Str(s='Assign - coins')), ctx=Load()), attr='append', ctx=Load()), args=[BinOp(left=Name(id='t1', ctx=Load()), op=Sub(), right=Name(id='t0', ctx=Load()))], keywords=[])), Assign(targets=[Name(id='t2', ctx=Store())], value=Call(func=Name(id='time', ctx=Load()), args=[], keywords=[])), Assi

In [31]:
%%time
for i in range(100):
    benchmark_calc_utility(deepcopy(game),100)

CPU times: user 4.57 s, sys: 32.7 ms, total: 4.6 s
Wall time: 4.65 s


## 4 second vs 14 seconds!

In [32]:
for key, val in performance.items():
    print(f'{key} = {np.sum(val)}')

Assign - coins = 0.001409292221069336
Assign - turn_prob_first = 1.0147919654846191
Assign - game_prob_first = 2.9141907691955566
Assign - winner_bets = 0.0016117095947265625
Assign - bet_tiles = 0.0018596649169921875
Expr - rename = 0.0013267993927001953
Assign - bets = 0.11316633224487305
Assign - multiply_array = 0.005779266357421875
Assign - bets_groupby = 0.00940847396850586
Assign - final = 0.33826398849487305
Assign - game_first = 0.05641627311706543
Assign - game_last = 0.0517888069152832
Assign - game_winner_other = 0.05377554893493652
Assign - game_loser_other = 0.05129075050354004
